In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import time, sys
import pandas as pd
import numpy as np
from fbprophet import Prophet
from tqdm import tqdm, tnrange
from multiprocessing import Pool, cpu_count
from functools import partial
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore", DeprecationWarning)
warnings.simplefilter("ignore", FutureWarning, )

future_preds = 28

In [ ]:




df_calendar = pd.read_csv("../input/kaggle-afc/calendar_afcs2020.csv")
df_train =  pd.read_csv("../input/kaggle-afc/sales_train_validation_afcs2020.csv",index_col=0)
evaluation = pd.read_csv("../input/kaggle-afc/sales_train_evaluation_afcs2020.csv")
df_train.index = df_train.index.str.replace('_validation', '')
submission = pd.read_csv("../input/kaggle-afc/sample_submission_afcs2020.csv")
day_cols = pd.Series([c for c in df_train.columns if c.find('d_')==0])




In [ ]:
df_train.head()

In [ ]:
def clean_data(df_train, day_cols, indx):
    t = df_train.loc[indx].copy()
    t.loc[day_cols[((t.loc[day_cols]>0).cumsum()==0).values]] = np.nan

    q1 = t.loc[day_cols].quantile(0.25)
    q3 = t.loc[day_cols].quantile(0.75)
    iqr = q3-q1
    qm = (q3+1.5*iqr)
    t.loc[day_cols][t.loc[day_cols]>qm] = qm
    return t

In [ ]:
df_calendar.index = df_calendar['d'].values
df_calendar['ds'] = pd.to_datetime(df_calendar['date'])
df_calendar['quarter'] = df_calendar['ds'].dt.quarter

In [ ]:
events1 = pd.Series(df_calendar['event_name_1'].values, index=df_calendar['ds'].values).dropna()
events2 = pd.Series(df_calendar['event_name_2'].values, index=df_calendar['ds'].values).dropna()
holidays = pd.DataFrame(pd.concat([events1, events2], axis=0))

In [ ]:
holidays['ds'] = holidays.index.values
holidays.rename({0: 'holiday'}, axis=1, inplace=True)
holidays.reset_index(drop=True, inplace=True)
del events1, events2
#%%
print('Cleaning data...', flush=True)
#data = [clean_data(df_train, day_cols, i) for i in tqdm(df_train.index)]
data = [clean_data(df_train, day_cols, i) for i in df_train.index]
df_train = pd.concat(data, axis=1).T

In [ ]:
print('Prepare prices')
df_sell_prices = pd.read_csv("../input/kaggle-afc/sell_prices_afcs2020.csv")

df_sell_prices['id'] = df_sell_prices['item_id'] + '_' + df_sell_prices['store_id']

df_sell_prices = df_sell_prices.pivot(index='id', columns='wm_yr_wk', values='sell_price')
df_sell_prices = df_sell_prices.fillna(method='bfill', axis=1)
df_prices = pd.DataFrame(index=df_train.index.values)
df_prices = list()
for i in df_sell_prices.columns:
    cols = df_calendar['d'][df_calendar['wm_yr_wk']==i]
    t = pd.concat([df_sell_prices[i] for j in cols], axis=1)
    t.columns = cols
    df_prices.append(t)
df_prices = pd.concat(df_prices, axis=1)

In [ ]:
def make_prediction(indx):
    global df_train, holidays, df_prices, df_calendar
    changepoints=list()
    uncertainty_samples=False
    changepoint_prior_scale=0.1
    changepoint_range=0.9
    holidays_prior_scale=10
    yearly_seasonality=2
    weekly_seasonality=1
    daily_seasonality=False
    monthly_fourier_order=None
    quarter_fourier_order=None
    seasonality_prior_scale=10
    seasonality_mode = 'additive'
    
    target = df_train.loc[indx, day_cols]

    df = df_calendar.iloc[:target.shape[0]+future_preds][['ds', 'month', 'wday', 'quarter']]
    df['y'] = target
    df['prices'] = df_prices.loc[indx].iloc[:df.shape[0]].values

    m = Prophet(growth='linear', uncertainty_samples=uncertainty_samples, changepoint_prior_scale=changepoint_prior_scale, changepoint_range=changepoint_range,
                holidays_prior_scale=holidays_prior_scale, yearly_seasonality=yearly_seasonality,
                daily_seasonality=daily_seasonality, weekly_seasonality=weekly_seasonality,
                holidays=holidays, seasonality_mode=seasonality_mode, seasonality_prior_scale=seasonality_prior_scale)

    if not monthly_fourier_order is None:
        m.add_seasonality(name='monthly', period=365.25/12, fourier_order=monthly_fourier_order)
    if not quarter_fourier_order is None:
        m.add_seasonality(name='quarterly', period=365.25/4, fourier_order=quarter_fourier_order)#, prior_scale=15)

    for reg in df.columns:
        if reg!='ds' and reg!='y':
            m.add_regressor(reg)
    m.fit(df.loc[target.loc[target.first_valid_index():].index])

    df.drop(['y'], axis=1, inplace=True)
    
    forecast = m.predict(df.iloc[-future_preds:])
    res = forecast['yhat']
    res.index = df.iloc[-future_preds:].index.values
    
    return res
#%%

In [ ]:
submission = pd.read_csv("../input/kaggle-afc/sample_submission_afcs2020.csv", index_col=0)
#%

print('Predicting...', flush=True)
start_time = time.time()
pool = Pool()

train_indxs = df_train.index

In [ ]:
res = pool.map(make_prediction, train_indxs)
pool.close()
pool.join()
end_time = time.time()
print('Exec speed=%.2f' %((end_time-start_time)/train_indxs.shape[0]))
#%%
for j, i in enumerate(train_indxs):
    submission.loc[i+'_validation'] = res[j].values
#%%
submission[submission<0]=0

In [ ]:
submission.shape

In [ ]:
submission.to_csv('submission123.csv')

In [ ]:


price_dfs = {'CA_3': price_df[price_df['store_id'] == 'CA_3']}



In [ ]:
price_dfs

In [ ]:
price_df = None

In [ ]:
store_id

In [ ]:
def transform_d_dates_to_dates(d_dates):
    return calender_df.set_index('d').loc[d_dates]['date']


def transform_dates_to_d_dates(dates):
    return calender_df.set_index('date').loc[dates]['d']


def transform_dates_to_wm_yr_wk(dates):
    return calender_df.set_index('date').loc[dates]['wm_yr_wk']


def get_avg_item_n_sold_prev_month(item_id, store_id, dates_df):
    assert len(item_id.split('_')) == 3
    assert store_id in ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
    assert dates_df.shape == (1604, 14)
    
    d_dates_month_ago = transform_dates_to_d_dates(dates_df['date'] - pd.DateOffset(days=28))
    assert d_dates_month_ago.shape == (dates_df.shape[0],)
    
    first_day = int(d_dates_month_ago.iloc[0].split('_')[1]) - 28
    last_day = int(d_dates_month_ago.iloc[-1].split('_')[1])
    assert first_day == 310 and last_day == 1941
    
    tmp = train_df[
        (train_df['item_id'] == item_id) &
        (train_df['store_id'] == store_id)
    ][['d_' + str(i) for i in range(first_day, last_day + 1)]]
    assert tmp.shape == (1, dates_df.shape[0] + 28)
    return tmp.iloc[0].rolling(28).mean().iloc[28:].to_numpy()


def get_item_n_sold_year_ago(item_id, store_id, dates_df):
    assert len(item_id.split('_')) == 3
    assert store_id in ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
    assert dates_df.shape == (1604, 14)
    
    d_dates_year_ago = transform_dates_to_d_dates(dates_df['date'] - pd.DateOffset(years=1))
    assert d_dates_year_ago.shape == (dates_df.shape[0],)
    
    tmp = train_df[
        (train_df['item_id'] == item_id) &
        (train_df['store_id'] == store_id)
    ][d_dates_year_ago]
    assert tmp.shape == (1, dates_df.shape[0])
    return tmp.iloc[0].to_numpy()


def get_item_price(item_id, store_id, dates_df):
    assert len(item_id.split('_')) == 3
    assert store_id in ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
    assert dates_df.shape == (1604, 14)
    
    wm_yr_wk = transform_dates_to_wm_yr_wk(dates_df['date']).to_numpy()
    assert wm_yr_wk.shape == (dates_df.shape[0],)
    
    week_to_price = price_dfs[store_id][
        (price_dfs[store_id]['item_id'] == item_id)
    ].set_index('wm_yr_wk')['sell_price'].to_dict()
    
    price = np.full(dates_df.shape[0], np.nan)
    for i in range(wm_yr_wk.shape[0]):
        week = wm_yr_wk[i]
        if week in week_to_price:
            price[i] = week_to_price[week]
    
    item_price_df = pd.DataFrame(data={'price': price})
    item_price_df = item_price_df.fillna(method='ffill').fillna(method='bfill')

    assert item_price_df['price'].isna().sum() == 0
    assert item_price_df.shape == (dates_df.shape[0], 1)

    # norm_price = item_price_df['price']
    # item_price_df['price'] /= np.linspace(1.00, 1.05, num=item_price_df.shape[0])  # inflation
    norm_price = item_price_df['price'].to_numpy()
    
    assert norm_price.shape == (dates_df.shape[0],)
    return norm_price


def get_is_snap(item_id, store_id, dates_df):
    assert 'FOODS' in item_id
    assert len(item_id.split('_')) == 3
    assert store_id in ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
    assert dates_df.shape == (1604, 14)
    
    if store_id in ['CA_1', 'CA_2', 'CA_3', 'CA_4']:
        return dates_df['snap_CA'].to_numpy()
    elif store_id in ['TX_1', 'TX_2', 'TX_3']:
        return dates_df['snap_TX'].to_numpy()
    elif store_id in ['WI_1', 'WI_2', 'WI_3']:
        return dates_df['snap_WI'].to_numpy()

    assert False
    return None
    
    
def get_week_days_features(item_id, store_id, dates_df):
    assert len(item_id.split('_')) == 3
    assert store_id in ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
    assert dates_df.shape == (1604, 14)
    
    return pd.DataFrame(
        index=dates_df['d'],
        data={
            'is_Monday': (dates_df['weekday'] == 'Monday').astype(int).to_numpy(),
            'is_Tuesday': (dates_df['weekday'] == 'Tuesday').astype(int).to_numpy(),
            'is_Wednesday': (dates_df['weekday'] == 'Wednesday').astype(int).to_numpy(),
            'is_Thursday': (dates_df['weekday'] == 'Thursday').astype(int).to_numpy(),
            'is_Friday': (dates_df['weekday'] == 'Friday').astype(int).to_numpy(),
            'is_Saturday': (dates_df['weekday'] == 'Saturday').astype(int).to_numpy()
        }
    )
    
    
def get_event_features(item_id, store_id, dates_df):
    assert len(item_id.split('_')) == 3
    assert store_id in ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
    assert dates_df.shape == (1604, 14)
    
    events_df = dates_df[['date', 'd', 'event_type_1', 'event_type_2']].copy()
    events_df['tomorrow_event_type_1'] = events_df['event_type_1'].shift(periods=-1)
    events_df['tomorrow_event_type_2'] = events_df['event_type_2'].shift(periods=-1)
    
    return pd.DataFrame(
        index=events_df['d'],
        data={
            'is_today_religious': (
                (events_df['event_type_1'] == 'Religious') |
                (events_df['event_type_2'] == 'Religious')
            ).astype(int).to_numpy(),
            'is_today_national': (
                (events_df['event_type_1'] == 'National') |
                (events_df['event_type_2'] == 'National')
            ).astype(int).to_numpy(),
            'is_today_cultural': (
                (events_df['event_type_1'] == 'Cultural') |
                (events_df['event_type_2'] == 'Cultural')
            ).astype(int).to_numpy(),
            'is_today_sporting': (
                (events_df['event_type_1'] == 'Sporting') |
                (events_df['event_type_2'] == 'Sporting')
            ).astype(int).to_numpy(),
            'is_tomorrow_religious': (
                (events_df['tomorrow_event_type_1'] == 'Religious') |
                (events_df['tomorrow_event_type_2'] == 'Religious')
            ).astype(int).to_numpy(),
            'is_tomorrow_national': (
                (events_df['tomorrow_event_type_1'] == 'National') |
                (events_df['tomorrow_event_type_2'] == 'National')
            ).astype(int).to_numpy(),
            'is_tomorrow_cultural': (
                (events_df['tomorrow_event_type_1'] == 'Cultural') |
                (events_df['tomorrow_event_type_2'] == 'Cultural')
            ).astype(int).to_numpy(),
            'is_tomorrow_sporting': (
                (events_df['tomorrow_event_type_1'] == 'Sporting') |
                (events_df['tomorrow_event_type_2'] == 'Sporting')
            ).astype(int).to_numpy()
        }
    )


def get_item_X_y(item, is_debug):
    assert len(item.split('_')) == 5
    
    dates_df = calender_df.iloc[365:]
    
    item_parts = item.split('_')
    item_id = item_parts[0] + '_' + item_parts[1] + '_' + item_parts[2]
    store_id = item_parts[3] + '_' + item_parts[4]
    
    df = pd.DataFrame(
        index=dates_df['d'].to_numpy(),
        data={
            'avg_item_n_sold_prev_month': get_avg_item_n_sold_prev_month(item_id, store_id, dates_df),
            'item_n_sold_year_ago': get_item_n_sold_year_ago(item_id, store_id, dates_df),
            'price': get_item_price(item_id, store_id, dates_df)
        }
    )
    
    if 'FOODS' in item:
        df['is_snap'] = get_is_snap(item_id, store_id, dates_df)
        
    df = pd.concat([
        df,
        get_week_days_features(item_id, store_id, dates_df),
        get_event_features(item_id, store_id, dates_df)
    ], axis=1)
    assert df.isna().sum().sum() == 0
    
    features_to_drop = []
    for feature in df.columns:
        if len(df[feature].unique()) <= 1:
            features_to_drop.append(feature)
    df = df.drop(features_to_drop, axis=1)
    if is_debug:
        print('Features', features_to_drop, 'have been dropped')
    
    target = train_df[
        train_df['id'] == item + '_evaluation'
    ][
        ['d_' + str(i) for i in range(366, 1942)]
    ].to_numpy()[0]
    assert target.shape == (1576,)
    target = np.concatenate([target, np.full(28, np.nan)])
    df['target'] = target

    return df

In [ ]:
def train_item_model(item, is_debug):
    df = get_item_X_y(item, is_debug)
    assert df.shape[0] == 1604

    X = df.drop(['target'], axis=1)
    y = df['target']
    
    X_train = X.loc[['d_' + str(i) for i in range(366, 1914)]]  # train
    y_train = y.loc[['d_' + str(i) for i in range(366, 1914)]]  # train
    assert X_train.shape[0] == y_train.shape[0] == 1548
    
    X_valid = X.loc[['d_' + str(i) for i in range(1914, 1942)]]  # public test
    y_valid = y.loc[['d_' + str(i) for i in range(1914, 1942)]]  # public test
    X_test = X.loc[['d_' + str(i) for i in range(1942, 1970)]]  # private test
    assert X_valid.shape[0] == y_valid.shape[0] == X_test.shape[0] == 28
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)  # train
    X_valid = scaler.transform(X_valid)  # public test
    X_test = scaler.transform(X_test)  # private test
    
    model = Ridge()
    model.fit(X_train, y_train)
    
    y_valid_pred = model.predict(X_valid)
    y_valid_pred[y_valid_pred < 0] = 0
    y_valid_pred[y_valid_pred > y.max()] = y.max()
    
    y_test_pred = model.predict(X_test)
    y_test_pred[y_test_pred < 0] = 0
    y_test_pred[y_test_pred > y.max()] = y.max()
    
    if is_debug:
        plot_feature_importances(model, X.columns)
        plot_public_test(y_valid, y_valid_pred)
        print('PUBLIC TEST: mean_absolute_error =', mean_absolute_error(y_valid, y_valid_pred))
    
    return y_valid_pred, y_test_pred

In [ ]:
get_item_X_y('HOBBIES_1_004_CA_1', is_debug=True)